This notebook guides through the process of semi-manually segmenting the background of the Transmission channel. The obtained mask sparsifies the image data significantly and is needed for computing astar distances through the microstructure. First load the Transmission channel into napari. Perform prewitt edge detection, gaussian denoising, binarize using otsu, run binary closing. Then comes the manual part where one needs to close all the edges. Once the has been done, use flood to segement the PDMS structure channels. Two masks are saved, one with the explant wel included, one with them excluded.

In [2]:
import napari
from tifffile import imread
import numpy as np

from skimage import filters
from skimage import morphology
from skimage.filters import threshold_otsu
from skimage.segmentation import flood

import pandas as pd

In [4]:
%gui qt

inp_path = '/run/media/loaloa/lbb_ssd/timelapse13_processed/'
inp_path = '/home/loaloa/Documents/timelapse13_processed/'

name = 'D06_G022'
# name = 'D15_G031'


fname = f'{name}_Transmission_compr.deflate.tif'
mask_fname = fname.replace('.tif', '_mask1.npy')
metadata_fname = fname.replace('.tif', '_metadata.csv')

tranl_file = inp_path+fname
transl_ch = imread(tranl_file)[0]

In [5]:
with napari.gui_qt():
    viewer = napari.Viewer()

In [6]:
# load transmission channel
viewer.add_image(transl_ch, name=name)
viewer.layers[name].blending='additive'
# viewer.layers['transl_ch'].contrast_limits = [.007*2**16, .0191*2**16]
viewer.layers[name].opacity = .7
viewer.layers[name].gamma = .72

## If old mask0 (or mask1) already exists, load it here. 


In [7]:
# mask1
which_mask = 'mask1'
mask_fname = f'{name}_Transmission_compr.deflate_{which_mask}.npy'
mask_file = inp_path+mask_fname

mask = np.load(mask_file)
viewer.add_labels(mask, name='inital_mask')

<Labels layer 'inital_mask' at 0x7fa822061ca0>

# Load from gimp segmentation 

In [49]:
# mask0
which_mask = 'mask0'
mask_fname = f'{name}_Transmission_compr.deflate_{which_mask}.tif'
mask_file = inp_path+mask_fname

mask = imread(mask_file)[:,:,0].astype(bool)
print(mask)
print(mask.shape)
viewer.add_labels(mask, name='inital_mask')

Shaped series: series shape does not match page shape
[[False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]
 ...
 [False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]]
(1972, 3868)


<Labels layer 'inital_mask' at 0x7f4cbda3c130>

In [68]:
# edge detection
prewitt = filters.prewitt(transl_ch)
viewer.add_image(prewitt, visible=False)
# perform guassian smoothing on edge filtered image
prewitt_gaussian = filters.gaussian(prewitt, sigma=1)
viewer.add_image(prewitt_gaussian, visible=False)
# binarize image
prewitt_bin = prewitt > threshold_otsu(prewitt_gaussian)
viewer.add_image(prewitt_bin, visible=False)
# Run this, then use the bucket tool to fill everything. 
# Play with sigma and solem diameter if things look bad.
# delete parts ofthe image where channels almost touch each other
diameter = 6
selem = morphology.selem.square(diameter) 
prewitt_bin_closed = morphology.binary_closing(prewitt_bin, selem)
viewer.add_labels(prewitt_bin_closed, name='inital_mask')

<Labels layer 'inital_mask' at 0x7fc687f08490>

In [51]:
# merge with gimp mask:
prewitt_bin_closed = mask | prewitt_bin_closed
viewer.add_labels(prewitt_bin_closed, name='inital_mask')


<Labels layer 'inital_mask [2]' at 0x7f4ceaaf9f40>

In [9]:
target = (0,1900)
notes = 'right container no channels'
design = int(name[1:3])
CLSM_area = int(name[5:8])
dt = 31 # in minutes
pixelsize = 0.62  #in um
# which_tl = 'tl14'
# incubation_time = 32*60   # aggrowell incubation, seeded incubation, imaged incubation
which_tl = 'tl`13'
incubation_time = 52*60


metadata = pd.Series({'target': target,
                      'notes': notes,
                      'design': design,
                      'CLSM_area': CLSM_area,
                      'dt': dt,
                      'which_tl': which_tl,
                      'incubation_time': incubation_time,
                      'pixelsize': pixelsize,
                })
                
metadata

target                               (0, 1900)
notes              right container no channels
design                                       6
CLSM_area                                   22
dt                                          31
which_tl                                 tl`13
incubation_time                           3120
pixelsize                                 0.62
dtype: object

In [70]:
# flood 
prewitt_bin_closed = viewer.layers['inital_mask'].data
print(prewitt_bin_closed.shape)
prewitt_bin_closed_flooded = flood(prewitt_bin_closed, target) 
# prewitt_bin_closed_flooded2 = flood(prewitt_bin_closed, (0,1720))
# prewitt_bin_closed_flooded = prewitt_bin_closed_flooded |prewitt_bin_closed_flooded2
viewer.add_labels(prewitt_bin_closed_flooded, name='before_clean')

(1972, 3868)


<Labels layer 'before_clean' at 0x7fc6cec5b2e0>

In [71]:
prewitt_bin_closed_flooded = viewer.layers['before_clean'].data
selem = morphology.selem.diamond(3)
final = filters.median(prewitt_bin_closed_flooded, selem)
# final fixing
viewer.add_labels(final, name='final')

<Labels layer 'final' at 0x7fc686578940>

------------------------------------

In [72]:
# final mask
mask = viewer.layers['final'].data.astype(bool)
# segment target and starts
viewer.add_labels(np.zeros_like(mask), name='goal_mask')
viewer.add_labels(np.zeros_like(mask), name='start_mask')

<Labels layer 'start_mask' at 0x7fc686578dc0>

In [73]:
goal_mask = viewer.layers['goal_mask'].data & mask
start_mask = viewer.layers['start_mask'].data & mask

In [74]:
np.save(tranl_file.replace('.tif', '_mask1.npy'), mask)
np.save(tranl_file.replace('.tif', '_mask1_goal.npy'), start_mask)
np.save(tranl_file.replace('.tif', '_mask1_start.npy'), goal_mask)
metadata.to_csv(inp_path+metadata_fname)

In [75]:
# check
mask = np.load(tranl_file.replace('.tif', '_mask1.npy'))
viewer.add_image(mask, opacity=.5)
goal_mask = np.load(tranl_file.replace('.tif', '_mask1_goal.npy'))
viewer.add_image(goal_mask, opacity=.5)
start_mask = np.load(tranl_file.replace('.tif', '_mask1_start.npy'))
viewer.add_image(start_mask, opacity=.5)

<Image layer 'start_mask [1]' at 0x7fc687c9f250>